# Análise e exploração de dados

Este ficheiro é para me "ambientar" com o que é pedido.
O "trabalho" oficial está no reposítório **git** da equipa: 
https://bitbucket.org/anadi-rita/anadi/src/main/

~~~bash
git clone git@bitbucket.org:anadi-rita/anadi.git
~~~

## Imports

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import utilidades as ut

## Variáveis

In [ ]:
proj_root_dir = Path(".").resolve().parent.parent
data_files_dir = "/ficheiros/Trabalho 1/"
ficheiro_dados_co = f"{proj_root_dir}{data_files_dir}CO_data.csv"
ficheiro_dicio_dados_co = f"{proj_root_dir}{data_files_dir}CO_data_dicionario.xlsx"

colunas_em_estudo = ["country", "year", "population", "co2"]

# Preparar uma lista de cores
cores = list(mcolors.BASE_COLORS.keys())


## Carregar os dados

In [ ]:
dados_co = pd.read_csv(
    ficheiro_dados_co,
    delimiter=",",
    header=0,
)

### Espreitar os dados

In [ ]:
dados_co.head()

## Carregar o dicionário

In [ ]:
dicionario_dados = pd.read_excel(ficheiro_dicio_dados_co)

## Análise e exploração de dados

Ponto 4.1 do trabalho

### Ponto 3
Construa um gráfico que permita comparar, no período 1900-2021, as emissões de CO2 per capita
de Portugal com a Espanha. 

#### Extrair os dados de Portugal e Espanha

Podemos fazer a seleção num único passo: 

In [ ]:
dados_co_pt_es_plt = dados_co.loc[dados_co.country.isin(["Portugal", "Spain"]), colunas_em_estudo]

Executar a linha abaixo, substitui os valores NaN por 0.
Tem efeitos no resultado final, dependendo do tipo de gráfico ou cálculos que se realizem.

In [ ]:
dados_co_pt_es_plt.fillna(0, inplace=True)  # Preencher os valores NaN com 0.

In [ ]:
dados_co_pt_es_plt.head()

In [ ]:
dados_co_pt_es_plt['co2_per_capita']=dados_co_pt_es_plt['co2'] * 1E6 / dados_co_pt_es_plt['population']

In [ ]:
dados_co_pt_es_plt.head()

In [ ]:
# Preparar os dados

In [ ]:
datas = dados_co_pt_es_plt.loc[dados_co_pt_es_plt.country == "Portugal", "year"].values
dados_pt = dados_co_pt_es_plt.loc[dados_co_pt_es_plt.country == "Portugal", "co2_per_capita"]
dados_es = dados_co_pt_es_plt.loc[dados_co_pt_es_plt.country == "Spain", "co2_per_capita"]


### Stack

Este gráfico pode ser interessante, é feito o "stack" dos valores, mas o resultado é "balanceado" à volta do 0.
Seria necessário trabalhar os eixos.

In [ ]:
stack_fig, stack_ax = plt.subplots(figsize=(12, 8))
# Configuraçao do ``Axes``.
stack_ax.set_title("Emissões de CO2 per capita entre 1900 e 2021")
stack_ax.set_xlabel("Ano")
stack_ax.set_ylabel(r"Emissão CO$_2$ per capita (toneladas)")
# stack_ax.grid(visible=True)
stack_ax.axhline(0)

stack_ax.stackplot(
    datas,  # Valores a utilizar no eixo dos XX
    [dados_pt, dados_es],  # Valores das áreas indexadas pelo ano.
    baseline="sym",
    labels=["Portugal", "Espanha"],  # O valor "humano" das etiquetas.
    colors=["green", "yellow"],  # Cores das áreas.
)
stack_ax.legend(loc="upper left")


### Plot

Apesar de simples, este é a meu ver o que mais facilmente permite ver a diferença entre os países. E assim responder mais adequadamente ao que se pede. Decididamente uma abordagem KISS (1).

(1) **K**eep **I**t **S**tupidly **S**imple - Esta versão é menos "ofensiva" 

In [ ]:
plot_fig, plot_ax = plt.subplots(figsize=(12, 8))
# Configuraçao do ``Axes``.
plot_ax.set_title(r"Emissões de CO$_2$ per capita entre 1900 e 2021")
plot_ax.set_xlabel("Ano")
plot_ax.set_ylabel(r"Emissão CO$_2$ per capita (toneladas)")

graficos = []
dados_para_grafico = [
    [dados_pt, "red", "Portugal"],
    [dados_es, "yellow", "Espanha"],
]

# Construir as várias "linhas"
for dados_pais in dados_para_grafico:
    grafico = plot_ax.plot(
        datas,  # Seria a utilizar para o eixo dos XX.
        dados_pais[0],  # Seria a utilizar para o eixo dos YY. 
        color=dados_pais[1],  # Cor da "linha".
        label=dados_pais[2],  # O valor "humano" das etiquetas. 
    )
    # Vou guardar as linhas numa lista caso queira mais tarde referênciar as mesmas.
    graficos.append(grafico)

plot_ax.legend(loc="upper left")

### Barras horizontais

Baseado em:
https://sharkcoder.com/data-visualization/mpl-bidirectional

Interessante, mas não ajuda a ver o que se pretende.


In [ ]:
bar_fig, bar_axes = plt.subplots(figsize=(12, 8), ncols=2, sharey=True)
bar_fig.tight_layout()
# Configuraçao do ``Axes``.
# bar_axes.set_title("Emissões de CO2 per capita entre 1900 e 2021")
# bar_axes.set_xlabel("Ano")
# bar_axes.set_ylabel(r"Emissão CO$_2$ per capita (toneladas)")
# bar_axes.grid(True)

# Inicializar bottom_series com uma Serie de zeros.
bottom_series = [0]*122
areas = []


# Construir as barras PT
barras_pt = bar_axes[0].barh(
    datas,  # Seria a utilizar para o eixo dos XX.
    dados_pt,
    color="yellow",  # Cor da "linha".
    label="Portugal",  # O valor "humano" das etiquetas. 
)
# Construir as barras PT
barras_es = bar_axes[1].barh(
    datas,  # Seria a utilizar para o eixo dos XX.
    dados_es,
    color="red",  # Cor da "linha".
    label="Espanha",  # O valor "humano" das etiquetas.
)
# bar_axes[0].invert_yxis() 
bar_axes[0].invert_xaxis()
# bar_axes[1].invert_xaxis()

bar_axes[1].legend(loc="lower right")
bar_axes[0].legend(loc="lower left")
bar_axes[1].grid(True)
bar_axes[0].grid(True)



